In [2]:
import pandas as pd
import numpy as np
import re, json

### Read data & Extract attributes

In [11]:
# read files: business, review, tip, user
business = pd.read_json("../data/business_city.json", lines = True)

In [12]:
steak_business = business[business["categories"].str.lower().str.contains("steakhouse") == True]
steak_business = steak_business[["business_id", "name", "latitude", "longitude", "stars", "attributes"]]

In [13]:
# one record contain NA in attributes, remove this record
steak_business = steak_business.dropna()
# reset index
steak_business = steak_business.reset_index().drop("index", axis = 1)

In [14]:
# extract attributes
attributes = []
for ind in steak_business.index: 
    attr = steak_business.loc[ind, "attributes"].keys()
    attributes.extend(list(attr))

attributes = sorted(list(set(attributes)))

In [15]:
attr_dict = {}
for attr in attributes: 
    attr_dict[attr] = []
for ind in steak_business.index:
    attribute = steak_business.loc[ind, "attributes"]
    for attr in attributes: 
        if attr in attribute.keys(): 
            attr_dict[attr].append(attribute[attr])
        else: 
            attr_dict[attr].append(np.nan)

In [16]:
attr_df = pd.DataFrame(attr_dict)
steak_attributes = pd.concat([steak_business, attr_df], axis = 1)

In [17]:
na_num = {}
for col in attr_df.columns: 
    na_num[col] = attr_df[col].isnull().sum()
extract_attr = []
for attr in list(na_num.keys()): 
    if na_num[attr] < 35: 
        extract_attr.append(attr)
extract_attr

['Alcohol',
 'Ambience',
 'BusinessAcceptsCreditCards',
 'BusinessParking',
 'GoodForKids',
 'HasTV',
 'OutdoorSeating',
 'RestaurantsAttire',
 'RestaurantsDelivery',
 'RestaurantsGoodForGroups',
 'RestaurantsPriceRange2',
 'RestaurantsReservations',
 'RestaurantsTakeOut']

In [19]:
# keep attributes with less missing proportion less than 10%
steak_attributes = steak_attributes[["business_id", "name", "latitude", "longitude", "stars", "Alcohol", 
                                     "Ambience", "BusinessAcceptsCreditCards", "BusinessParking", "GoodForKids", 
                                     "HasTV", "OutdoorSeating", "RestaurantsAttire", "RestaurantsDelivery", 
                                     "RestaurantsGoodForGroups", "RestaurantsPriceRange2", 
                                     "RestaurantsReservations", "RestaurantsTakeOut"]]

### Summary of each attributes

#### Alcohol

In [20]:
steak_attributes.Alcohol.value_counts()

u'full_bar'         254
'full_bar'           52
u'none'              10
'none'                7
u'beer_and_wine'      6
'beer_and_wine'       1
Name: Alcohol, dtype: int64

#### BusinessAcceptsCreditCards

In [21]:
steak_attributes.BusinessAcceptsCreditCards.value_counts()

True     344
None       1
False      1
Name: BusinessAcceptsCreditCards, dtype: int64

Most steakhouse businesses have BusinessAcceptsCreditCards attribute. omit!

#### GoodForKids

In [22]:
steak_attributes.GoodForKids.value_counts()

True     232
False    103
None       1
Name: GoodForKids, dtype: int64

#### HasTV

In [23]:
steak_attributes.HasTV.value_counts()

True     306
False     28
Name: HasTV, dtype: int64

#### OutdoorSeating

In [24]:
steak_attributes.OutdoorSeating.value_counts()

False    195
True     142
Name: OutdoorSeating, dtype: int64

#### RestaurantsAttire

In [25]:
steak_attributes.RestaurantsAttire.value_counts()

u'casual'    183
'casual'     100
u'dressy'     28
'dressy'      23
Name: RestaurantsAttire, dtype: int64

#### RestaurantsDelivery

In [26]:
steak_attributes.RestaurantsDelivery.value_counts()

False    258
True      81
None       2
Name: RestaurantsDelivery, dtype: int64

#### RestaurantsGoodForGroups

In [27]:
steak_attributes.RestaurantsGoodForGroups.value_counts()

True     332
False      7
Name: RestaurantsGoodForGroups, dtype: int64

Most steakhouse business have RestaurantsGoodForGroups attribute. omit!

#### RestaurantsPriceRage2

In [28]:
steak_attributes.RestaurantsPriceRange2.value_counts()

2    228
3     78
1     22
4     12
Name: RestaurantsPriceRange2, dtype: int64

#### RestaruantsReservations

In [29]:
steak_attributes.RestaurantsReservations.value_counts()

True     250
False     93
None       1
Name: RestaurantsReservations, dtype: int64

#### RestaurantsTakeOut

In [30]:
steak_attributes.RestaurantsTakeOut.value_counts()

True     299
False     35
Name: RestaurantsTakeOut, dtype: int64

Only retain  Alcohol, Ambience, BusinessParking, GoodForKids, HasTV, OutdoorSeating, RestaurantsAttire, Restaurant Delivery, RestaurantsPriceRange2, RestaurantsReservations, RestaurantsTakeOut. 

In [31]:
steak_attributes = steak_attributes.drop(["BusinessAcceptsCreditCards", "RestaurantsGoodForGroups"], axis = 1)

In [32]:
# Not able to impute missing values, omit imcomplete records
steak_mask_amb = steak_attributes.mask(steak_attributes["Ambience"].eq("None"))
steak_attributes = steak_mask_amb.dropna().reset_index().drop("index", axis = 1)

### Extract information from Ambience

expand Ambience information

In [ ]:
ambience = ["romantic", "intimate", "classy", 'upscale', "touristy", "trendy", "casual", "divey", "hipster"]
ambiences = {}

for amb in sorted(ambience): 
    keys = "Ambience." + amb
    ambiences[keys] = []
    
for ind in steak_attributes.index: 
    amb = steak_attributes.loc[ind, "Ambience"]
    amb_false = re.sub('False|None', '0', amb)
    amb_true = re.sub('True', '1', amb_false)
    amb_punc = re.sub('\'', '\"', amb_true)
    amb_dict = json.loads(amb_punc)
    for amb_type in ambience: 
        keys = "Ambience." + amb_type
        if amb_type in amb_dict.keys(): 
            ambiences[keys].append(amb_dict[amb_type])
        else: 
            ambiences[keys].append(0)
            
steak_ambience = pd.DataFrame(ambiences)

In [ ]:
steak_attributes = pd.concat([steak_attributes, steak_ambience], axis = 1)

### Extract information from BusinessParking

expand BusinessParking information

In [ ]:
steak_attributes.loc[0, "BusinessParking"]

In [ ]:
park_type = ["garage", "street", "validated", "lot", "valet"]
parkings = {}

for park in sorted(park_type): 
    keys = "BusinessParking." + park
    parkings[keys] = []
    
for ind in steak_attributes.index: 
    parking = steak_attributes.loc[ind, "BusinessParking"]
    if parking != "None": 
        parking_false = re.sub('False|None', '0', parking)
        parking_true = re.sub('True', '1', parking_false)
        parking_punc = re.sub('\'', '\"', parking_true)
        parking_dict = json.loads(parking_punc)
        for p in park_type: 
            keys = "BusinessParking." + p
            if p in parking_dict.keys(): 
                parkings[keys].append(parking_dict[p])
            else: 
                parkings[keys].append(0)
    else: 
        for p in parkings.keys(): 
            parkings[p].append(0)

steak_parking = pd.DataFrame(parkings)

In [ ]:
steak_attributes = pd.concat([steak_attributes, steak_parking], axis = 1)

In [ ]:
# drop original "Ambience", "BusinessParking"
steak_attributes = steak_attributes.drop(["Ambience", "BusinessParking"], axis = 1)
steak_attributes.head()

In [ ]:
steak_attributes.to_csv("../data/steak_attributes.csv", index = False)